<a href="https://colab.research.google.com/github/comeddy/langchain-guide/blob/main/Langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LangChain이란?
* Language : Large Language Model의 Language
* Chain: 모듈들을 체인으로 연결하여 다른 툴의 출력을 다음 툴의 입력으로 연결하는 방식
  * 예: 검색 -> OpenAI(LLM) -> 계산
* LLM(Large Language Model)을 이용해서 뭔가 만들때 쓸수 있는 툴/라이브러리 모음.
* 파이썬과 자바스크립트 라이브러리를 제공.
* 검색, 메모리, 계산, 요약, 색인, Agent(에이전트) 등의 기능을 제공
* 모듈화로 쉽게 부품을 갈아끼울수 있음
* LLM은 기본 OpenAI (text-davinci-003) 이용

---
참고 자료:
* 파이썬 문서: https://python.langchain.com/en/latest/index.html
* 개념 문서: https://docs.langchain.com/docs/
* 소스: https://github.com/hwchase17/langchain

* 설명 유튜브 영상: https://youtu.be/KdbPZNdFJU0
---

In [1]:
#@title 기본 패키지(openai, langchain) 설치
!pip install openai
!pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 4.1 MB/s eta 0:00:00


In [2]:
#@title 기타 패키지 설치 (구글검색, 위키피디아, VectorStore, HuggingFace Embedding)
!pip install google-search-results
!pip install wikipedia
!pip install faiss-cpu # 오픈소스 벡터DB (Facebook, MIT license)
!pip install sentence_transformers # HuggingFace Embedding 사용 위해서 필요
!pip install tiktoken # Summarization 할때 필요

  Preparing metadata (setup.py) ... done
  Created wheel for google-search-results: filename=google_search_results-2.4.2-py3-none-any.whl size=32004 sha256=ef52adcc55f655f373cadea1a89c61e3e2d2e18b2317b5306fc80a8398cb927a
  Stored in directory: /root/.cache/pip/wheels/d3/b2/c3/03302d12bb44a2cdff3c9371f31b72c0c4e84b8d2285eeac53
Successfully built google-search-results
  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11680 sha256=56bd8f4ce3bb36559c437e97413f5d2eb4181a6fa2f2dedde6669776d91d6809
  Stored in directory: /root/.cache/pip/wheels/5e/b6/c5/93f3dec388ae76edc830cb42901bb0232504dfc0df02fc50de
Successfully built wikipedia
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 33.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━

In [7]:
#@title 0. API 키 설정
import os
#@markdown https://platform.openai.com/account/api-keys
OPENAI_API_KEY = "sk-xxx" #@param {type:"string"}

#@markdown https://huggingface.co/settings/tokens
#@markdown HuggingFace에서 모델 다운로드나 클라우드 모델 사용하기 위해서 필요 (무료)
HUGGINGFACEHUB_API_TOKEN = "xxx" #@param {type:"string"}

#@markdown https://serpapi.com/manage-api-key
#@markdown 구글 검색하기 위해서 필요 (월 100회 무료)
SERPAPI_API_KEY = "xx" #@param {type:"string"}

os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
os.environ["HUGGINGFACEHUB_API_TOKEN"] = HUGGINGFACEHUB_API_TOKEN
os.environ["SERPAPI_API_KEY"] = SERPAPI_API_KEY


In [8]:
#@title 1. OpenAI LLM (text-davinci-003)

from langchain.llms import OpenAI

llm = OpenAI(model_name='text-davinci-003', temperature=0.9)

llm('1980년대 메탈 음악 5곡 추천해줘.')

'\n\n1. Iron Maiden – The Trooper\n2. Judas Priest – Breaking the Law\n3. Testament – Practice What You Preach\n4. Metallica – Enter Sandman\n5. Pantera – Cowboys from Hell'

In [10]:
#@title 2. ChatOpenAI LLM (gpt-3.5-turbo)

from langchain.chat_models import ChatOpenAI
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

chat = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0.9)
sys = SystemMessage(content="당신은 음악 추천을 해주는 전문 AI입니다.")
msg = HumanMessage(content='1980년대 메탈 음악 5곡 추천해줘.')

aimsg = chat([sys, msg])
aimsg.content

'1980년대 메탈 음악은 많은 명곡들이 있지만, 다음은 그 중 5곡을 추천해 드리겠습니다:\n\n1. Iron Maiden - "The Number of the Beast"\n2. Metallica - "Master of Puppets"\n3. Black Sabbath - "Iron Man"\n4. Judas Priest - "Breaking the Law"\n5. Guns N\' Roses - "Sweet Child O\' Mine"\n\n이 추천 곡들은 1980년대 메탈 음악의 대표적인 곡들 중 일부입니다. 각각의 곡들은 그들만의 독특한 사운드와 가사로 인정받고 있으며, 이 시기의 메탈 음악을 대표하는 명곡들로 손꼽힙니다. 즐겁게 감상하시길 바랍니다!'

In [11]:
#@title 3. Prompt Template & chain

from langchain.prompts import PromptTemplate

prompt = PromptTemplate(
    input_variables=["상품"],
    template="{상품} 만드는 회사 이름 추천해줘. 기억에 남는 한글 이름으로",
)

prompt.format(상품="AI 여행 추천 서비스")

'AI 여행 추천 서비스 만드는 회사 이름 추천해줘. 기억에 남는 한글 이름으로'

In [14]:
from langchain.chains import LLMChain
chain = LLMChain(llm=chat, prompt=prompt)

# chain.run("AI 여행 추천 서비스")
chain.run(상품="AI 여행 추천 서비스")

'1. 별빛여행\n2. 봄날여행\n3. 햇살가득한여행\n4. 달빛여행\n5. 소망여행\n6. 푸른바다여행\n7. 청춘여행\n8. 미소여행\n9. 황금여행\n10. 즐거운여행'

In [15]:
#@title 4. ChatPromptTemplate & chain

from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

chat = ChatOpenAI(temperature=0)

template="You are a helpful assisstant that tranlates {input_language} to {output_language}."
system_message_prompt = SystemMessagePromptTemplate.from_template(template)
human_template="{text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])


chatchain = LLMChain(llm=chat, prompt=chat_prompt)
chatchain.run(input_language="English", output_language="Korean", text="I love programming.")


'나는 프로그래밍을 사랑합니다.'

In [16]:
#@title 5. Agents and Tools

from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType

* Tool: 특정 일을 하는 역할. 예) 구글 검색, 디비 조회, Python 실행/계산.
* Agents: 어떤 툴을 어떤 순서로 실행할지 결정하는 역할 (LLM 이용)

In [17]:
# tools = load_tools(["serpapi", "llm-math"], llm=chat)
tools = load_tools(["wikipedia", "llm-math"], llm=chat)

agent = initialize_agent(tools, llm=chat, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

In [18]:
agent.run("페이스북 창업자는 누구인지? 그의 현재(2023년) 나이를 제곱하면?")



> Entering new AgentExecutor chain...
I need to find out who the founder of Facebook is and then calculate their current age squared.
Action: Wikipedia
Action Input: "founder of Facebook"
Observation: Page: Facebook
Summary: Facebook is an online social media and social networking service owned by American technology giant Meta Platforms. Created in 2004 by Mark Zuckerberg with fellow Harvard College students and roommates Eduardo Saverin, Andrew McCollum, Dustin Moskovitz, and Chris Hughes, its name derives from the face book directories often given to American university students. Membership was initially limited to only Harvard students, gradually expanding to other North American universities and, since 2006, anyone over 13 years old. As of December 2022, Facebook claimed 2.96 billion monthly active users, and ranked third worldwide among the most visited websites. It was the most downloaded mobile app of the 2010s.Facebook can be accessed from devices with Internet connectivity,

'The founder of Facebook is Mark Zuckerberg. His current age squared is 41^2 = 1681.'

In [19]:
agent.tools

[WikipediaQueryRun(name='Wikipedia', description='A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.', args_schema=None, return_direct=False, verbose=False, callbacks=None, callback_manager=None, tags=None, metadata=None, handle_tool_error=False, api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from '/usr/local/lib/python3.10/dist-packages/wikipedia/__init__.py'>, top_k_results=3, lang='en', load_all_available_meta=False, doc_content_chars_max=4000)),
 Tool(name='Calculator', description='Useful for when you need to answer questions about math.', args_schema=None, return_direct=False, verbose=False, callbacks=None, callback_manager=None, tags=None, metadata=None, handle_tool_error=False, func=<bound method Chain.run of LLMMathChain(memory=None, callbacks=None, callback_manager=None, verbose=False, tags=None, metadata=None, llm_chain=

In [20]:
print(agent.tools[0].description)
print(agent.tools[1].description)

A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.
Useful for when you need to answer questions about math.


In [21]:
#@title 6. Memory
from langchain import ConversationChain

conversation = ConversationChain(llm=chat, verbose=True)
conversation.predict(input="인공지능에서 Transformer가 뭐야?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: 인공지능에서 Transformer가 뭐야?
AI:

> Finished chain.


'Transformer는 자연어 처리 분야에서 매우 중요한 모델입니다. 이 모델은 주로 기계 번역, 질의응답, 요약 등의 작업에 사용됩니다. Transformer는 2017년에 발표된 논문 "Attention is All You Need"에서 처음 소개되었습니다. 이 모델은 기존의 순차적인 방식이 아닌, 전체 문장을 동시에 처리하는 방식을 사용하여 성능을 크게 향상시켰습니다. Transformer는 주로 어텐션 메커니즘을 사용하여 입력 문장의 각 단어가 출력에 얼마나 중요한지를 결정합니다. 이를 통해 문장의 의미를 잘 이해하고 번역, 요약 등의 작업을 수행할 수 있습니다.'

In [22]:
conversation.predict(input="RNN하고 차이 설명해줘.")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: 인공지능에서 Transformer가 뭐야?
AI: Transformer는 자연어 처리 분야에서 매우 중요한 모델입니다. 이 모델은 주로 기계 번역, 질의응답, 요약 등의 작업에 사용됩니다. Transformer는 2017년에 발표된 논문 "Attention is All You Need"에서 처음 소개되었습니다. 이 모델은 기존의 순차적인 방식이 아닌, 전체 문장을 동시에 처리하는 방식을 사용하여 성능을 크게 향상시켰습니다. Transformer는 주로 어텐션 메커니즘을 사용하여 입력 문장의 각 단어가 출력에 얼마나 중요한지를 결정합니다. 이를 통해 문장의 의미를 잘 이해하고 번역, 요약 등의 작업을 수행할 수 있습니다.
Human: RNN하고 차이 설명해줘.
AI:

> Finished chain.


'RNN은 순환 신경망(Recurrent Neural Network)의 약자로, 순차적인 데이터를 처리하는 데에 주로 사용되는 모델입니다. RNN은 이전의 입력에 의존하여 현재의 출력을 계산하는 구조를 가지고 있습니다. 이전의 정보를 기억하고 현재의 입력과 함께 고려하여 다음 출력을 예측하는 데에 적합한 모델입니다.\n\n반면에 Transformer는 RNN과는 다른 구조를 가지고 있습니다. Transformer는 기존의 순차적인 방식이 아닌, 전체 문장을 동시에 처리하는 방식을 사용합니다. 이를 위해 어텐션 메커니즘을 사용하여 입력 문장의 각 단어가 출력에 얼마나 중요한지를 결정합니다. 이러한 구조를 통해 Transformer는 RNN보다 더 긴 문장을 처리하는 데에 효과적이며, 자연어 처리 분야에서 많은 성능 향상을 이끌어냈습니다.'

In [23]:
conversation.memory

ConversationBufferMemory(chat_memory=ChatMessageHistory(messages=[HumanMessage(content='인공지능에서 Transformer가 뭐야?', additional_kwargs={}, example=False), AIMessage(content='Transformer는 자연어 처리 분야에서 매우 중요한 모델입니다. 이 모델은 주로 기계 번역, 질의응답, 요약 등의 작업에 사용됩니다. Transformer는 2017년에 발표된 논문 "Attention is All You Need"에서 처음 소개되었습니다. 이 모델은 기존의 순차적인 방식이 아닌, 전체 문장을 동시에 처리하는 방식을 사용하여 성능을 크게 향상시켰습니다. Transformer는 주로 어텐션 메커니즘을 사용하여 입력 문장의 각 단어가 출력에 얼마나 중요한지를 결정합니다. 이를 통해 문장의 의미를 잘 이해하고 번역, 요약 등의 작업을 수행할 수 있습니다.', additional_kwargs={}, example=False), HumanMessage(content='RNN하고 차이 설명해줘.', additional_kwargs={}, example=False), AIMessage(content='RNN은 순환 신경망(Recurrent Neural Network)의 약자로, 순차적인 데이터를 처리하는 데에 주로 사용되는 모델입니다. RNN은 이전의 입력에 의존하여 현재의 출력을 계산하는 구조를 가지고 있습니다. 이전의 정보를 기억하고 현재의 입력과 함께 고려하여 다음 출력을 예측하는 데에 적합한 모델입니다.\n\n반면에 Transformer는 RNN과는 다른 구조를 가지고 있습니다. Transformer는 기존의 순차적인 방식이 아닌, 전체 문장을 동시에 처리하는 방식을 사용합니다. 이를 위해 어텐션 메커니즘을 사용하여 입력 문장의 각 단어가 출력에 얼마나 중요한지를 결정합니다. 이러한 구조를 통해 Transformer는 RNN보다 더 긴 문장을 

* ConversationBufferMemory : 대화 기록(기본)
* ConversationBufferWindowMemory : 마지막 n개의 대화만 기억
* Entity Memory : 개체에 대한 정보를 저장
* Conversation Knowledge Graph Memory: 개체의 triple 저장: (sam, 좋아하는 색, 파랑)
* ConversationSummaryMemory : 대화의 요약본을 저장
* ConversationSummaryBufferMemory : 대화 요약본 + 마지막 n토큰 기억
* ConversationTokenBufferMemory : 마지막 n토큰 기억
* VectorStore-Backed Memory : 벡터DB에 정보 저장

In [24]:
#@title 7. Document Loaders
from langchain.document_loaders import WebBaseLoader

loader = WebBaseLoader(web_path="https://ko.wikipedia.org/wiki/NewJeans")

documents = loader.load()

In [25]:
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)
len(docs)

# 4096 token = 3000 English word

9

In [26]:
print(docs[1].page_content)

EP에 대해서는 New Jeans (EP) 문서를 참고하십시오.
NewJeans뉴진스기본 정보결성 지역대한민국 서울특별시장르K-pop댄스 팝활동 시기2022년 7월 22일 ~ 현재레이블하이브소속사어도어웹사이트공식 홈페이지구성원민지하니다니엘해린혜인
NewJeans(뉴진스)는 2022년 7월 22일에 데뷔한 대한민국의 5인조 걸 그룹으로, 소속사는 하이브 산하의 레이블인 어도어이다. SM 엔터테인먼트 비주얼 디렉터 출신으로 하이브에 영입된 민희진이 프로듀서로 나서서 발굴한 걸 그룹이다. 2022년 8월 18일에 《엠카운트다운》에서 데뷔 3주만에 첫 1위에 올랐다. 2023년 8월에 잼버리 폐회식 K-POP 공연에 출연했다.


구성원[편집]


이름

소개


민지


본명: 김민지(金玟池)
생년월일: 2004년 5월 7일(2004-05-07)(19세)
학력 : 범어고등학교 (졸업)
출생지: 대한민국 강원도 춘천시
활동기간 : 2022년 7월 22일 ~ 현재


하니


본명: 하니 팜(Hanni Pham), 팜 응옥 헌(范玉欣/Phạm Ngọc Hân)
생년월일: 2004년 10월 6일(2004-10-06)(18세)
출생지: 오스트레일리아 빅토리아주 멜버른
활동기간 : 2022년 7월 22일 ~ 현재


다니엘


본명: 다니엘 마쉬(Danielle Marsh), 모지혜(牟智慧)
생년월일: 2005년 4월 11일(2005-04-11)(18세)
출생지: 오스트레일리아 뉴사우스웨일스주 뉴캐슬
활동기간 : 2022년 7월 22일 ~ 현재


해린


본명: 강해린(姜諧潾)
생년월일: 2006년 5월 15일(2006-05-15)(17세)
출생지: 대한민국 서울특별시
학력: 중학교 졸업학력 검정고시 (합격)
활동기간: 2022년 7월 22일 ~ 현재


혜인


본명: 이혜인(李惠仁)
생년월일: 2008년 4월 21일(2008-04-21)(15세)
출생지: 대한민국 인천광역시 미추홀구
학력: 한국국제크리스천스쿨 (중퇴)
활동기간: 2022년 7월 22일 ~ 현재


---
* Web Page
* PDF
* Email
* Twitter
* DataFrame (pandas)
* YouTube
* Notion
* Google Drive
* Powerpoint, Word,
* ...
---

In [27]:
#@title 8. Summarization
from langchain.chains.summarize import load_summarize_chain
chain = load_summarize_chain(chat, chain_type="map_reduce", verbose=True)
chain.run(docs[:3])




> Entering new MapReduceDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Write a concise summary of the following:


"NewJeans - 위키백과, 우리 모두의 백과사전


본문으로 이동

주 메뉴

주 메뉴
사이드바로 이동
숨기기

		둘러보기
	


대문최근 바뀜요즘 화제임의의 문서로기부

		사용자 모임
	


사랑방사용자 모임관리 요청

		편집 안내
	


도움말정책과 지침질문방

언어

이 위키백과에서 언어 링크는 문서 제목 건너편의 문서 최상단에 있습니다. 최상단으로 이동합니다.

검색

검색


계정 만들기로그인


개인 도구

 계정 만들기 로그인

		로그아웃한 편집자를 위한 문서 더 알아보기

기여토론

목차
사이드바로 이동
숨기기


처음 위치

1구성원

2음반

음반 하위섹션 토글하기

2.1EP

2.2싱글

3음반 외 활동

4수상 목록

수상 목록 하위섹션 토글하기

4.1가요 프로그램 1위

5기타 시상식

6각주

7외부 링크

목차 토글

목차 토글

NewJeans

25개 언어


العربيةБеларускаяDeutschEnglishEsperantoEspañolEuskaraفارسیSuomiFrançaisՀայերենBahasa Indonesia日本語PolskiPortuguêsРусскийSlovenčinaСрпски / srpskiSvenskaไทยTürkçeУкраїнськаOʻzbekcha / ўзбекчаTiếng Việt中文

링크 편집

문서토론

한국어

읽기편집역사 보기

도구

도구
사이드바로 이동
숨기기

		동작
	


읽기편집역사 보기

		일반
	


여기를 가리키는 문서가리키는 글의 최근 바뀜파일 올리기특수 문서 목록고유 링크문서 정보이 문서 인용하기위키데이터 항목

		인쇄/내보내기
	


책 만들기PDF로 다운로드인쇄용 판

		다른

'NewJeans is a 5-member girl group from South Korea signed under the label "어도어." They achieved their first number one on "M Countdown" just three weeks after their debut. The group has released two EPs and several singles, and has won multiple awards on music programs.'

* "stuff": LLM 한번에 다 보냄. 길면 오류남
* "map_reduce": 나눠서 요약, 전체 요약본 다시 요약
* "refine": (요약 + 다음 문서) => 요약
* "map_rerank": 점수매겨서 중요한거로 요약

In [28]:
#@title 9. Embeddings and VectorStore
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.embeddings import OpenAIEmbeddings

# embeddings = OpenAIEmbeddings()
embeddings = HuggingFaceEmbeddings()

from langchain.indexes import VectorstoreIndexCreator
from langchain.vectorstores import FAISS

# from langchain.text_splitter import RecursiveCharacterTextSplitter
# text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)

index = VectorstoreIndexCreator(
    vectorstore_cls=FAISS,
    embedding=embeddings,
    # text_splitter=text_splitter,
    ).from_loaders([loader])

# 파일로 저장
index.vectorstore.save_local("faiss-nj")

* Embedding : LLM에 input(text)를 넣었을때 나오는 특징 벡터 - Neural Network의 특정 메모리를 읽어서 만들고, 보통 512개 이상의 float로 만들어짐.
* 의미상 비슷한 input이 들어갔을때, 나오는 특징 벡터가 유사한 특징을 갖고, 거리상 가깝다.
* OpenAI에서 text를 넣으면 Embedding을 돌려주는 API를 제공한다. (다만 비용 발생)
* 로컬에서 유사한 알고리즘을 돌릴수 있다. (다만 똑같은 벡터는 아니고, openAI 임베딩과는 비교할수 없다)
* HuggingFaceEmbeddings 사용하면 로컬에서 돌릴수 있고, GPU 있다면 가속도 지원함.
* Vector Database는 Embedding 벡터와 텍스트를 저장하는 DB
  * Pinecone 서비스 (유료, 무료 제한적)
  * FAISS (페이스북, 오픈소스, 로컬)
  * Embeddings -> 텍스트를 저장
  * Embeddings로 검색할수 있으며, 거리상 가장 가까운 항목을 가져올수 있음. (거리를 구하는 방법은 여러가지)
  * 유사한 의미를 갖는 텍스트에 대한 Embedding 벡터는 거리상 가깝다.

In [29]:
index.query("뉴진스의 데뷔곡은?", llm=chat, verbose=True)



> Entering new RetrievalQA chain...

> Finished chain.


'뉴진스의 데뷔곡은 "Attention"입니다.'

In [30]:
index.query("뉴진스의 데뷔 멤버는?", llm=chat, verbose=True)



> Entering new RetrievalQA chain...

> Finished chain.


'뉴진스(NewJeans)의 데뷔 멤버는 다음과 같습니다:\n\n1. 민지 (김민지)\n2. 하니 (하니 팜)\n3. 다니엘 (다니엘 마쉬)\n4. 해린 (강해린)\n5. 혜인 (이혜인)\n\n이 다섯 명이 뉴진스의 데뷔 멤버입니다.'

In [31]:
index.query("멤버의 나이는?", llm=chat, verbose=True)



> Entering new RetrievalQA chain...

> Finished chain.


'민지: 19세\n하니: 18세\n다니엘: 18세\n해린: 17세\n혜인: 15세'

In [32]:
index.query("멤버의 나이는? (오늘은 2023년 4월 18일)", llm=chat, verbose=True)



> Entering new RetrievalQA chain...

> Finished chain.


'현재 날짜가 2023년 4월 18일이므로, 멤버들의 나이는 다음과 같습니다:\n\n- 강해린(해린): 17세 (2006년 5월 15일 출생)\n- 이혜인(혜인): 15세 (2008년 4월 21일 출생)\n- 김민지(민지): 19세 (2004년 5월 7일 출생)\n- 하니(하니 팜): 18세 (2004년 10월 6일 출생)\n- 다니엘(다니엘 마쉬): 18세 (2005년 4월 11일 출생)'

In [33]:
#@title FAISS 벡터DB 디스크에서 불러오기
from langchain.indexes.vectorstore import VectorStoreIndexWrapper

fdb = FAISS.load_local("faiss-nj", embeddings)
index2 = VectorStoreIndexWrapper(vectorstore=fdb)

In [34]:
index2.query("뉴진스의 데뷔 멤버는?", llm=chat, verbose=True)



> Entering new RetrievalQA chain...

> Finished chain.


'뉴진스(NewJeans)의 데뷔 멤버는 다음과 같습니다:\n\n1. 민지 (김민지)\n2. 하니 (하니 팜)\n3. 다니엘 (다니엘 마쉬)\n4. 해린 (강해린)\n5. 혜인 (이혜인)\n\n이 다섯 명이 뉴진스의 데뷔 멤버입니다.'